In [1]:
import nltk
from string import punctuation
from nltk.tokenize import RegexpTokenizer,word_tokenize
from unicodedata import normalize
from nltk.stem import RSLPStemmer
from nltk.util import ngrams
import pandas as pd

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('rslp')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Flavio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Flavio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\Flavio\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [2]:
import time

In [3]:
df_assunto= pd.read_table("dados-conle-anonimizado-assunto-notnull - dados-conle-anonimizado-assunto-notnull.tsv",sep=";")

In [4]:
arr_assunto = df_assunto.to_numpy()
y,X = arr_assunto[:,0],arr_assunto[:,1]
y = [i.strip() for i in y]

In [5]:
base = pd.read_csv("base_20230428_douglas.csv", delimiter=',', encoding="utf-8")

In [6]:
from pyserini.search import SimpleSearcher

In [7]:
def verificar(y,top_n,top_k=20):
    for i,n in enumerate(top_n):
        id = n.docid
        
        name = base.loc[int(id),"txtNome"]
        if str(name).strip()==str(y).strip():
            return 1
        if i==(top_k-1):
            break
    return 0

In [8]:
def avaliacaoRecall(isPreprocess,top_k=20):
    quant_encontrado = 0
    quant_relevante =0
    antes = time.time()
    for l,x in zip(y,X):

        tokenized_query3 = x
        if isPreprocess:
            tokenized_query3 = preprocess(x)
    

        top_n = searcher.search(tokenized_query3)
    
        quant_relevante+=1
        quant_encontrado+=verificar(l,top_n,top_k)
    
    recall = quant_encontrado / quant_relevante
    depois = time.time()
    print("R@20: "+str(recall))
    print("Duração: %f" %(depois - antes))

In [9]:
def avaliacaoRR(isPreprocess,top_k=20):
    quant=0
    for l,x in zip(y,X):

        tokenized_query3 = x
        if isPreprocess:
            tokenized_query3 = preprocess(x)
    
        top_n = searcher.search(tokenized_query3)
        if len(top_n)>0:
            name = base.loc[int(top_n[0].docid),"txtNome"]
            if str(name).strip()==str(l).strip():
                quant+=1
        else:
            quant+=0
    rr = quant/295
    print("RR: %f" %(rr))

In [10]:
def avaliacaoPrecision(isPreprocess,top_k=20):
    soma=0
    for l,x in zip(y,X):

        tokenized_query3 = x
        if isPreprocess:
            tokenized_query3 = preprocess(x)
    
        top_n = searcher.search(tokenized_query3)
    
        encontrado=verificar(l,top_n,top_k)
        soma+= (encontrado / top_k)
    precision = soma / 295
    print("P@ %d: %f " % (top_k,precision))

In [11]:
def avaliacaoMAP(isPreprocess,top_k=20):
    l_v = list()
    labels_nsL = list()
    for l,x in zip(y,X):

        tokenized_query3 = x
        if isPreprocess:
            tokenized_query3 = preprocess(x)
    
        top_n = searcher.search(tokenized_query3)
        l_v.append(l)
        temp = list()
        for i,n in enumerate(top_n):
            id = n.docid
            name = base.loc[int(id),"txtNome"]
            temp.append(name)
            if i==(top_k-1):
                break
            
        
        labels_nsL.append(temp)
        
    denominador = 0
    encontrou=False
    soma=0
    for Y,x in zip(l_v,labels_nsL):
        calc = 0
        quant=0
        for k in x:
            if str(k).strip() == Y.strip():
                quant=1
                encontrou=True
            denominador+=1
            calc=quant/denominador
            if encontrou:
                calc = calc * 1
            else:
                calc = calc * 0
            soma+=calc
            encontrou = False
    MAP = soma /295
    print("MAP: %f" % (MAP))

In [10]:
class Savoy:

    def __removeAllPTAccent(self, old_word):
        word = list(old_word)
        len_word = len(word)-1
        for i in range(len_word, -1, -1):
            if word[i] == 'ä':
                word[i] = 'a'
            if word[i] == 'â':
                word[i] = 'a'
            if word[i] == 'à':
                word[i] = 'a'
            if word[i] == 'á':
                word[i] = 'a'
            if word[i] == 'ã':
                word[i] = 'a'
            if word[i] == 'ê':
                word[i] = 'e'
            if word[i] == 'é':
                word[i] = 'e'
            if word[i] == 'è':
                word[i] = 'e'
            if word[i] == 'ë':
                word[i] = 'e'
            if word[i] == 'ï':
                word[i] = 'i'
            if word[i] == 'î':
                word[i] = 'i'
            if word[i] == 'ì':
                word[i] = 'i'
            if word[i] == 'í':
                word[i] = 'i'
            if word[i] == 'ü':
                word[i] = 'u'
            if word[i] == 'ú':
                word[i] = 'u'
            if word[i] == 'ù':
                word[i] = 'u'
            if word[i] == 'û':
                word[i] = 'u'
            if word[i] == 'ô':
                word[i] = 'o'
            if word[i] == 'ö':
                word[i] = 'o'
            if word[i] == 'ó':
                word[i] = 'o'
            if word[i] == 'ò':
                word[i] = 'o'
            if word[i] == 'õ':
                word[i] = 'o'
            if word[i] == 'ç':
                word[i] = 'c'

        new_word = "".join(word)
        return new_word

    def __finalVowelPortuguese(self, word):
        len_word = len(word)
        if len_word > 3:
            if word[-1] == 'e' or word[-1] == 'a' or word[-1] == 'o':
                word = word[:-1]

        return word

    def __remove_PTsuffix(self, word):
        len_word = len(word)

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'e' and (word[-3] == 'r' or word[-3] == 's' or word[-3] == 'z' or word[-3] == 'l'):
                word = word[:-2]
                return word
        if len_word > 2:
            if word[-1] == 's' and word[-2] == 'n':
                new_word = list(word)
                new_word[-2] = 'm'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if (word[-1] == 's' and word[-2] == 'i') and (word[-3] == 'e' or word[-3] == 'é'):
                new_word = list(word)
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i' and word[-3] == 'a':
                new_word = list(word)
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i' and word[-3] == 'ó':
                new_word = list(word)
                new_word[-3] = 'o'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i':
                new_word = list(word)
                new_word[-1] = 'l'
                sing = "".join(new_word)
                return sing

        if len_word > 2:
            if word[-1] == 's' and word[-2] == 'e' and word[-3] == 'õ':
                new_word = list(word)
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing
            if word[-1] == 's' and word[-2] == 'e' and word[-3] == 'ã':
                new_word = list(word)
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 5:
            if word[-1] == 'e' and word[-2] == 't' and word[-3] == 'n' and word[-4] == 'e' and word[-5] == 'm':
                word = word[:-5]
                return word

        if len_word > 2:
            if word[-1] == 's':
                word = word[:-1]

        return word

    def __normFemininPortuguese(self, word):

        len_word = len(word)

        if len_word < 3 or word[-1] != 'a':
            return word

        if len_word > 6:

            if word[-2] == 'h' and word[-3] == 'n' and word[-4] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'c' and word[-3] == 'a' and word[-4] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'r' and word[-3] == 'i' and word[-4] == 'e':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

        if len_word > 5:
            if word[-2] == 'n' and word[-3] == 'o':
                new_word = list(word)
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                masc = "".join(new_word)
                masc = masc[:-1]
                return masc

            if word[-2] == 'r' and word[-3] == 'o':
                word = word[:-1]
                return word

            if word[-2] == 's' and word[-3] == 'o':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 's' and word[-3] == 'e':
                new_word = list(word)
                new_word[-3] = 'ê'
                masc = "".join(new_word)
                masc = masc[:-1]
                return masc

            if word[-2] == 'c' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'd' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'd' and word[-3] == 'a':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'v' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'm' and word[-3] == 'a':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'n':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

        return word

    def stem(self, word):
        len_word = len(word)
        if len_word > 2:
            word = self.__remove_PTsuffix(word)
            word = self.__normFemininPortuguese(word)
            word = self.__finalVowelPortuguese(word)
            word = self.__removeAllPTAccent(word)

        return word

In [11]:
class RSLP_S:
    def __plural_reduction(self, word):
        excep = ["lápis","cais","mais","crúcis","biquínis","pois","depois","dois","leis" ]
        excep_s = ["aliás","pires","lápis","cais","mais","mas","menos", "férias","fezes","pêsames","crúcis","gás", "atrás","moisés","através","convés","ês", "país","após","ambas","ambos","messias"]

        len_word = len(word)
        new_word = list(word)

        if len_word >= 3:
            if new_word[-1] == 's' and new_word[-2] == 'n':
                new_word[-2] = 'm'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'õ':
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return  sing

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'ã':
                if word == 'mães':
                    word = word[:-1]
                    return word
                else:
                    new_word[-2] = 'o'
                    sing = "".join(new_word)
                    sing = sing[:-1]
                    return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'a':
                if word != 'cais' and word != 'mais':
                    new_word[-2] = 'l'
                    sing = "".join(new_word)
                    sing = sing[:-1]
                    return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'é':
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'e':
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'ó':
                new_word[-3] = 'o'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'i':
                if word not in excep:
                    new_word[-1] = 'l'
                    sing = "".join(new_word)
                    return sing

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'l':
                word = word[:-2]
                return word

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'r':
                word = word[:-2]
                return word

            if new_word[-1] == 's':
                if word not in excep_s:
                    word = word[:-1]

        return word

    def stem(self, word):
        word = self.__plural_reduction(word)

        return word

## Config 1

In [12]:
index_path = 'indexE/sem_pre_processamento'

In [13]:
searcher = SimpleSearcher(index_path)
searcher.set_bm25(k1=1.5,b=0.75)

SimpleSearcher class has been deprecated, please use LuceneSearcher from pyserini.search.lucene instead


In [14]:
def preprocess(txt):
    txt = str(txt)
    terms = word_tokenize(txt)
    terms = " ".join(terms)
    return terms

## Avaliação

### Recall

In [17]:
avaliacaoRecall(True,5)

R@20: 0.13220338983050847
Duração: 1.419490


In [18]:
avaliacaoRecall(True,10)

R@20: 0.17966101694915254
Duração: 0.747000


In [15]:
avaliacaoRecall(True,20)

R@20: 0.31864406779661014
Duração: 3.140032


### Precision

In [20]:
avaliacaoPrecision(True,5)

P@ 5: 0.026441 


In [21]:
avaliacaoPrecision(True,10)

P@ 10: 0.017966 


In [22]:
avaliacaoPrecision(True,20)

P@ 20: 0.008983 


### MAP

In [23]:
avaliacaoMAP(True,5)

MAP: 0.000873


In [24]:
avaliacaoMAP(True,10)

MAP: 0.000505


In [25]:
avaliacaoMAP(True,20)

MAP: 0.000505


### RR

In [26]:
avaliacaoRR(True,5)

RR: 0.071186


## Config 5

In [16]:
index_path = 'indexE/pre_processamento_lower_remocao_punctuation_acentuacao_stopwords'

In [17]:
searcher = SimpleSearcher(index_path)
searcher.set_bm25(k1=1.5,b=0.75)

SimpleSearcher class has been deprecated, please use LuceneSearcher from pyserini.search.lucene instead


In [18]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]
    terms = " ".join(terms)
    return terms

## Avaliação

### Recall

In [30]:
avaliacaoRecall(True,5)

R@20: 0.2
Duração: 0.550018


In [31]:
avaliacaoRecall(True,10)

R@20: 0.2440677966101695
Duração: 0.601024


In [19]:
avaliacaoRecall(True,20)

R@20: 0.34576271186440677
Duração: 0.744996


### Precision

In [33]:
avaliacaoPrecision(True,5)

P@ 5: 0.040000 


In [34]:
avaliacaoPrecision(True,10)

P@ 10: 0.024407 


In [35]:
avaliacaoPrecision(True,20)

P@ 20: 0.012203 


### MAP

In [36]:
avaliacaoMAP(True,5)

MAP: 0.001152


In [37]:
avaliacaoMAP(True,10)

MAP: 0.000664


In [38]:
avaliacaoMAP(True,20)

MAP: 0.000664


### RR

In [39]:
avaliacaoRR(True,5)

RR: 0.108475


## Config 8

In [20]:
index_path = 'indexE/pre_processamento_rslp_full'

In [21]:
searcher = SimpleSearcher(index_path)
searcher.set_bm25(k1=1.5,b=0.75)

SimpleSearcher class has been deprecated, please use LuceneSearcher from pyserini.search.lucene instead


In [22]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

## Avaliação

### Recall

In [43]:
avaliacaoRecall(True,5)

R@20: 0.38305084745762713
Duração: 1.423002


In [44]:
avaliacaoRecall(True,10)

R@20: 0.4406779661016949
Duração: 1.422359


In [23]:
avaliacaoRecall(True,20)

R@20: 0.4847457627118644
Duração: 1.576999


### Precision

In [46]:
avaliacaoPrecision(True,5)

P@ 5: 0.076610 


In [47]:
avaliacaoPrecision(True,10)

P@ 10: 0.044068 


In [48]:
avaliacaoPrecision(True,20)

P@ 20: 0.022034 


### MAP

In [49]:
avaliacaoMAP(True,5)

MAP: 0.001850


In [50]:
avaliacaoMAP(True,10)

MAP: 0.001012


In [51]:
avaliacaoMAP(True,20)

MAP: 0.001012


### RR

In [52]:
avaliacaoRR(True,5)

RR: 0.233898


## Config 9

In [24]:
index_path = 'indexE/pre_processamento_savoy_full'

In [25]:
searcher = SimpleSearcher(index_path)
searcher.set_bm25(k1=1.5,b=0.75)

SimpleSearcher class has been deprecated, please use LuceneSearcher from pyserini.search.lucene instead


In [26]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

## Avaliação

### Recall

In [56]:
avaliacaoRecall(True,5)

R@20: 0.3423728813559322
Duração: 0.870002


In [57]:
avaliacaoRecall(True,10)

R@20: 0.4101694915254237
Duração: 0.858035


In [27]:
avaliacaoRecall(True,20)

R@20: 0.4440677966101695
Duração: 1.007250


### Precision

In [59]:
avaliacaoPrecision(True,5)

P@ 5: 0.068475 


In [60]:
avaliacaoPrecision(True,10)

P@ 10: 0.041017 


In [61]:
avaliacaoPrecision(True,20)

P@ 20: 0.020508 


### MAP

In [62]:
avaliacaoMAP(True,5)

MAP: 0.001775


In [63]:
avaliacaoMAP(True,10)

MAP: 0.001008


In [64]:
avaliacaoMAP(True,20)

MAP: 0.001008


### RR

In [65]:
avaliacaoRR(True,5)

RR: 0.200000


## Config 18

In [28]:
index_path = 'indexE/pre_processamento_unigram_bigram_rslp_full'

In [29]:
searcher = SimpleSearcher(index_path)
searcher.set_bm25(k1=1.5,b=0.75)

SimpleSearcher class has been deprecated, please use LuceneSearcher from pyserini.search.lucene instead


In [30]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)

## Avaliação

### Recall

In [69]:
avaliacaoRecall(True,5)

R@20: 0.423728813559322
Duração: 1.604999


In [70]:
avaliacaoRecall(True,10)

R@20: 0.46440677966101696
Duração: 1.655754


In [31]:
avaliacaoRecall(True,20)

R@20: 0.49491525423728816
Duração: 1.661852


### Precision

In [72]:
avaliacaoPrecision(True,5)

P@ 5: 0.084746 


In [73]:
avaliacaoPrecision(True,10)

P@ 10: 0.046441 


In [74]:
avaliacaoPrecision(True,20)

P@ 20: 0.023220 


### MAP

In [75]:
avaliacaoMAP(True,5)

MAP: 0.001940


In [76]:
avaliacaoMAP(True,10)

MAP: 0.001062


In [77]:
avaliacaoMAP(True,20)

MAP: 0.001062


### RR

In [78]:
avaliacaoRR(True,5)

RR: 0.257627


## Config 21

In [32]:
index_path = 'indexE/pre_processamento_unigram_bigram_savoy_full'

In [33]:
searcher = SimpleSearcher(index_path)
searcher.set_bm25(k1=1.5,b=0.75)

SimpleSearcher class has been deprecated, please use LuceneSearcher from pyserini.search.lucene instead


In [34]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)

## Avaliação

### Recall

In [82]:
avaliacaoRecall(True,5)

R@20: 0.3694915254237288
Duração: 0.960965


In [83]:
avaliacaoRecall(True,10)

R@20: 0.4271186440677966
Duração: 0.965000


In [35]:
avaliacaoRecall(True,20)

R@20: 0.4576271186440678
Duração: 1.054125


### Precision

In [85]:
avaliacaoPrecision(True,5)

P@ 5: 0.073898 


In [86]:
avaliacaoPrecision(True,10)

P@ 10: 0.042712 


In [87]:
avaliacaoPrecision(True,20)

P@ 20: 0.021356 


### MAP

In [88]:
avaliacaoMAP(True,5)

MAP: 0.001810


In [89]:
avaliacaoMAP(True,10)

MAP: 0.001007


In [90]:
avaliacaoMAP(True,20)

MAP: 0.001007


### RR

In [91]:
avaliacaoRR(True,5)

RR: 0.230508


## Config 22

In [36]:
index_path = 'indexE/pre_processamento_rslps_full'

In [37]:
searcher = SimpleSearcher(index_path)
searcher.set_bm25(k1=1.5,b=0.75)

SimpleSearcher class has been deprecated, please use LuceneSearcher from pyserini.search.lucene instead


In [38]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    
    return " ".join(terms)

## Avaliação

### Recall

In [95]:
avaliacaoRecall(True,5)

R@20: 0.21016949152542372
Duração: 0.564994


In [96]:
avaliacaoRecall(True,10)

R@20: 0.2576271186440678
Duração: 0.548001


In [39]:
avaliacaoRecall(True,20)

R@20: 0.4745762711864407
Duração: 0.825439


### Precision

In [98]:
avaliacaoPrecision(True,5)

P@ 5: 0.042034 


In [99]:
avaliacaoPrecision(True,10)

P@ 10: 0.025763 


In [100]:
avaliacaoPrecision(True,20)

P@ 20: 0.012881 


### MAP

In [101]:
avaliacaoMAP(True,5)

MAP: 0.001163


In [102]:
avaliacaoMAP(True,10)

MAP: 0.000674


In [103]:
avaliacaoMAP(True,20)

MAP: 0.000674


### RR

In [104]:
avaliacaoRR(True,5)

RR: 0.101695


## Config 23

In [40]:
index_path = 'indexE/pre_processamento_unigram_bigram_rslps_full'

In [41]:
searcher = SimpleSearcher(index_path)
searcher.set_bm25(k1=1.5,b=0.75)

SimpleSearcher class has been deprecated, please use LuceneSearcher from pyserini.search.lucene instead


In [42]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)

## Avaliação

### Recall

In [108]:
avaliacaoRecall(True,5)

R@20: 0.26101694915254237
Duração: 0.686034


In [109]:
avaliacaoRecall(True,10)

R@20: 0.3050847457627119
Duração: 0.693036


In [43]:
avaliacaoRecall(True,20)

R@20: 0.4542372881355932
Duração: 0.932535


### Precision

In [111]:
avaliacaoPrecision(True,5)

P@ 5: 0.052203 


In [112]:
avaliacaoPrecision(True,10)

P@ 10: 0.030508 


In [113]:
avaliacaoPrecision(True,20)

P@ 20: 0.015254 


### MAP

In [114]:
avaliacaoMAP(True,5)

MAP: 0.001371


In [115]:
avaliacaoMAP(True,10)

MAP: 0.000774


In [116]:
avaliacaoMAP(True,20)

MAP: 0.000774


### RR

In [117]:
avaliacaoRR(True,5)

RR: 0.132203
